In [15]:
import bs4
import os
from dotenv import load_dotenv
# 올바른 임포트 경로로 수정
from langchain_community.document_loaders import PyMuPDFLoader, WebBaseLoader  # PyPDFLoader 대신 사용
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

# 환경 변수 설정
os.environ['OPENAI_API_KEY'] = openai_api_key




In [18]:
# LOADING
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)

docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)


# INDEXING
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())




In [19]:
# retriever
chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'fetch_k': 4}
)

In [27]:
from langchain_classic.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 2

print("type of bm25", type(bm25_retriever))

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.2, 0.8]
)

# 예시 고객 문의
query = "향후 집값에 대해서 알려줘"

# 관련 문서/제품 검색
docs = ensemble_retriever.invoke(query)

# 각 문서에서 페이지 내용을 추출하여 출력
for doc in docs:
    print(doc)


docs


type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>
page_content='올해 서울 전셋집 잡기 전략 고강도 대출 규제에 전세물량 줄어 올해 서울 전셋값 4.7% 상승 전망 새 아파트 평년대비 반토막인데 서초서 5천가구 넘게 풀려 눈길 은평엔 2500가구 대단지 들어서 실수요자 현장 찾아


매일경제

1시간전' metadata={'source': 'https://news.naver.com/section/101'}
page_content='트럼프 미국 대통령이 한국 제품에 대한 관세 인상을 위협하는 가운데 현대차 호세 무뇨스 사장이 월스트리트저널과 인터뷰에서 대미 투자 의지를 거듭 확인했습니다. 월스트리트저널은 무뇨스 사장이 향후 4년간 260억 달러


YTN' metadata={'source': 'https://news.naver.com/section/101'}
page_content='정부가 ‘지역별 전기요금’(일명 지역별 차등전기요금) 도입 방안을 연내 확정한다. 또 올해 1분기(1~3월)에 산업용 전기요금 체계 개편이 추진된다. 기후에너지환경부는 1일 이 같은 내용의 ‘에너지 전환 분야 업무계


부산일보

3시간전' metadata={'source': 'https://news.naver.com/section/101'}


[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='올해 서울 전셋집 잡기 전략 고강도 대출 규제에 전세물량 줄어 올해 서울 전셋값 4.7% 상승 전망 새 아파트 평년대비 반토막인데 서초서 5천가구 넘게 풀려 눈길 은평엔 2500가구 대단지 들어서 실수요자 현장 찾아\n\n\n매일경제\n\n1시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='트럼프 미국 대통령이 한국 제품에 대한 관세 인상을 위협하는 가운데 현대차 호세 무뇨스 사장이 월스트리트저널과 인터뷰에서 대미 투자 의지를 거듭 확인했습니다. 월스트리트저널은 무뇨스 사장이 향후 4년간 260억 달러\n\n\nYTN'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='정부가 ‘지역별 전기요금’(일명 지역별 차등전기요금) 도입 방안을 연내 확정한다. 또 올해 1분기(1~3월)에 산업용 전기요금 체계 개편이 추진된다. 기후에너지환경부는 1일 이 같은 내용의 ‘에너지 전환 분야 업무계\n\n\n부산일보\n\n3시간전')]

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = """
    당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버젼을 생성하는 것입니다. 사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다. 각 질문은 새 줄로 구분하여 제공하세요 . 원본질문: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)